# Magic in Numpy
> Fast and Useful Matrix Operations and Linear Algebra

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]


## Numpy Basics

We will now go over some basic approaches on a seemingly simple matrix for illustrative purposes. Hopefully some of the efficient and useful properties of Numpy become apparent.

A favorite work by a favorite artist, Dürer's *Melencolia I* (1514) includes sophisticated use of mathematical allegory, particularly in the top-right corner.

<img src="melancholia.jpg" width="400"> 


# <img src="melancholia_detail.jpg" width="200">
The matrix is thus:


In [5]:
import numpy as np

X = np.array([[16, 3,  2, 13],
              [5, 10, 11, 8],
              [9,  6,  7, 12],
              [4, 15, 14,  1]])
print(X)

[[16  3  2 13]
 [ 5 10 11  8]
 [ 9  6  7 12]
 [ 4 15 14  1]]


In [6]:
type(X)

numpy.ndarray

## Magic Squares

This matrix is purported to be a magic square. We must fit the following constraints:

1) Magic
2) Square

Simple enough. Starting with the second condition, Numpy provides a number of methods. Though magic cubes and tesseracts are possible, we can begin with a square. Here's a simple function to detect if an array is square.

In [9]:
def is_square(M: np.ndarray) -> bool:
    '''
    Arguments: M, a 2-d matrix
    Returns: a boolean, True if square
    '''
    assert M.ndim == 2
    return True if M.shape[0] == M.shape[1] else False

is_square(X)

True

## Vectorized Summation: Magic

Now, the more involved condition. A square is "magic" iff the array exhibits the following properties:

i) Each of the $n$ elements are of the set of distinct positive integers $[1,2,3,...,n^2]$, such that $n$ is the "order" of the square. Dürer thus presents a $4^{th}$ order magic square.

ii) The sum of the $n$ numbers about any horizontal, vertical or main diagnonal is the same number – the "magic" constant. It is known that such magic constants can be given by $ \mathcal{M}(X_n) = \frac{1}{n}\sum_{k=1}^{n^2}k = \frac{1}{2}n(n^2+1)$ .


Aside:
iii) The complement to a magic square is derived from subtracting every number in a given magic square by $n^2 + 1$.

Back to Dürer. We can check each condition as follows.

In [113]:
def is_magic(M:np.ndarray)->bool:

    #By constraints i) & ii)
    assert M.shape[0] == M.shape[1], 'Not a square.'

    n = M.shape[0]

    assert np.array_equal(np.sort(M.flatten()), np.arange(n**2) + 1), 'Expected elements from [1,2,...,n^2]'

    column_sums = np.sum(M,axis=0)
    #Note that summing across axis 0 actually returns column-wise sums, and vice-versa.
    row_sums = np.sum(M, axis=1)

    diagonal_sums = np.array([np.trace(M),np.trace(np.fliplr(M))]).astype(int)

    # print(diagonal_sums)

    magic_num_sum = np.unique(np.concatenate( (column_sums,row_sums,diagonal_sums) ))


    if len(magic_num_sum) == 1:
        print(f'Magic number is {magic_num_sum} with order {n}.')
        return True

In [109]:
#note:
np.fliplr(X).diagonal().sum() == np.flipud(X).diagonal().sum()

True

In [110]:
#Also note:
np.trace(X) == np.diagonal(X).sum()

True

In [134]:
is_magic(X)

Magic number is [34] with order 4.


True

In [135]:
X

array([[16,  3,  2, 13],
       [ 5, 10, 11,  8],
       [ 9,  6,  7, 12],
       [ 4, 15, 14,  1]])

## Fast Indexing: Gnomon Magic

Dürer's square is actually a Gnomonic Magic Square – that is, each non-overlapping root subsquare bordering the four sides of the square ($2\times 2$ subsquare), as well as the central subsquare, sums to the magic constant of the overarching square.

The Gnomon is the portion of the sundial casting a shadow. In a way we also cast a magic projection on subarrays of our main magic square.

We can verify this easily – in Numpy, arrays can be efficiently split with minimal logic, rather than looping over each element and hard-indexing.


In [218]:
a,b,c,d = [quadrant for sub_x in np.split(X,2, axis = 0) for quadrant in np.split(sub_x,2, axis = 1)]

n = X.shape[0]

n_subsquare = np.sqrt(n).astype(int)
start = n//2 - (n_subsquare // 2)
end = n//2 + (n_subsquare // 2)

e = X[start:end,start:end]

sections = [a, b, c, d, e]
sections

[array([[16,  3],
        [ 5, 10]]),
 array([[ 2, 13],
        [11,  8]]),
 array([[ 9,  6],
        [ 4, 15]]),
 array([[ 7, 12],
        [14,  1]]),
 array([[10, 11],
        [ 6,  7]])]

In [221]:
print(set([sum(s.flatten()) for s in sections]))

{34}


All quadrants sum to the magic number of 34. As such, we have verified the deliberate style of Dürer.

# Linear Algebra